In [1]:
import zipfile
with zipfile.ZipFile("/content/drive/MyDrive/pneumonia dataset.zip","r") as zip_ref:
    zip_ref.extractall("targetdir")

In [1]:
from tensorflow.keras.applications import VGG16
conv_base = VGG16(weights='imagenet',
include_top=False,
input_shape=(224, 244, 3))

In [2]:
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
train_datagen = ImageDataGenerator(
rescale=1./255,
rotation_range=40,
width_shift_range=0.2,
height_shift_range=0.2,
shear_range=0.2,
zoom_range=0.2,
horizontal_flip=True,
fill_mode='nearest')
test_datagen = ImageDataGenerator(rescale=1./255)

In [3]:
train_dir = '/content/targetdir/pneumonia dataset/train'
train_generator = train_datagen.flow_from_directory(
train_dir,
target_size=(224, 224),
batch_size=20,
class_mode='binary')

Found 3059 images belonging to 2 classes.


In [4]:
val_dir = '/content/targetdir/pneumonia dataset/val'

In [5]:
validation_generator = test_datagen.flow_from_directory(
val_dir,
target_size=(224, 224),
batch_size=20,
class_mode='binary')

Found 16 images belonging to 2 classes.


In [6]:
test_dir = '/content/targetdir/pneumonia dataset/test'

In [7]:
test_generator = test_datagen.flow_from_directory(test_dir, target_size=(224,224),batch_size=20,class_mode='binary')

Found 624 images belonging to 2 classes.


In [8]:
conv_base.trainable = True
set_trainable = False
for layer in conv_base.layers:
  if layer.name == 'block5_conv1':
    set_trainable = True
  if set_trainable:
    layer.trainable = True
  else:
    layer.trainable = False

In [9]:
from keras import models
from keras import layers
model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [10]:
from keras import optimizers
import tensorflow
opt = tensorflow.keras.optimizers.RMSprop(learning_rate=2e-5)
model.compile(loss='binary_crossentropy',
optimizer=opt,
metrics=['acc'])

In [11]:
history = model.fit_generator(
train_generator,
steps_per_epoch=90,
epochs=30,
validation_data=validation_generator,
validation_steps=1)

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/30
90/90 [==============================] - 79s 708ms/step - loss: 0.3502 - acc: 0.8455 - val_loss: 0.3522 - val_acc: 0.8125
Epoch 2/30
90/90 [==============================] - 52s 575ms/step - loss: 0.2215 - acc: 0.9156 - val_loss: 0.4757 - val_acc: 0.8125
Epoch 3/30
90/90 [==============================] - 51s 568ms/step - loss: 0.1818 - acc: 0.9311 - val_loss: 0.3463 - val_acc: 0.9375
Epoch 4/30
90/90 [==============================] - 52s 577ms/step - loss: 0.1867 - acc: 0.9255 - val_loss: 0.3234 - val_acc: 0.8750
Epoch 5/30
90/90 [==============================] - 51s 562ms/step - loss: 0.1463 - acc: 0.9411 - val_loss: 0.4271 - val_acc: 0.8125
Epoch 6/30
90/90 [==============================] - 51s 568ms/step - loss: 0.1366 - acc: 0.9444 - val_loss: 0.6847 - val_acc: 0.6875
Epoch 7/30
90/90 [==============================] - 51s 568ms/step - loss: 0.1360 - acc: 0.9477 - val_loss: 0.4342 - val_acc: 0.8125
Epoch 8/30
90/90 [==============================] - 52s 573ms/step - 

In [12]:
model.evaluate(test_generator)


32/32 [==============================] - 11s 329ms/step - loss: 0.2268 - acc: 0.9215


[0.2268095761537552, 0.9214743375778198]

In [13]:
model.save('Fine Tuned Pneumonia Classification Model.h5')